# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [13]:
def expected_return(portfolio_data, risk_free_rate, market_risk_premium):
    total_invested = portfolio_data['Amount Invested'].sum()

    expected_return = 0

    for index, investment in portfolio_data.iterrows():
        weight = investment['Amount Invested'] / total_invested
        individual_return = risk_free_rate + (investment['Beta'] * market_risk_premium)
        contribution_to_return = weight * individual_return
        
        expected_return += contribution_to_return

    return expected_return


Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [17]:
def volatility(portfolio_data, correlation):
    total_invested = portfolio_data['Amount Invested'].sum()

    volatility = 0

    for index, investment in portfolio_data.iterrows():
        weight = investment['Amount Invested'] / total_invested
        stock_volatility = investment['Volatility']
        volatility += weight**2 * stock_volatility**2

    additional = len(portfolio_data) * correlation

    for index, investment in portfolio_data.iterrows():
        additional *= investment['Volatility'] * (investment['Amount Invested'] / total_invested)

    volatility += additional

    volatility = np.sqrt(volatility)

    return volatility


Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [19]:
def expected_return2(risk_free_rate, market_risk_premium, market_volatility, portfolio_volatility):
    
    volatility_ratio = portfolio_volatility / market_volatility

    expected_return = risk_free_rate + (volatility_ratio * market_risk_premium)

    return expected_return

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [31]:
def CAPM_beta(return_path, factors_path):
    returns = pd.read_csv(return_path)
    returns['date'] = returns['year'].astype(str) + returns['month'].astype(str).str.zfill(2)
    returns['date'] = returns['date'].astype(int)

    factors = pd.read_csv(factors_path)

    merged = pd.merge(returns, factors, on='date')
    merged[['Mkt-RF', 'RF']] /= 100

    exp_returns = merged.groupby('CUSIP')['RET'].mean().reset_index(name='Exp-Ret')

    covariances = (
        merged.groupby('CUSIP')
        .apply(lambda row: np.cov((row['RET'] - row['RF']), row['Mkt-RF'])[1, 0])
        .reset_index(name='Cov')
    )
    variances = merged.groupby('CUSIP')['Mkt-RF'].var().reset_index(name='Var')

    merged_cov_var = pd.merge(covariances, variances, on='CUSIP')
    merged_cov_var['Beta'] = merged_cov_var['Cov'] / merged_cov_var['Var']

    result = pd.merge(exp_returns, merged_cov_var, on='CUSIP')

    output = result.rename(columns={'Exp-Ret': 'EXP_RETURN', 'Beta': 'BETA'})[['CUSIP', 'EXP_RETURN', 'BETA']]

    return output


Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [34]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    val = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    return val[1]